# wPLI

In [1]:
# imports
from pathlib import Path
import re

import numpy as np
import pandas as pd
from scipy.io import loadmat

import mne
from mne_connectivity import spectral_connectivity_time


# MATLAB
## Prepare raw data for MATLAB

In [ ]:
target_ch = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6',
             'P1', 'P2', 'P3', 'P4', 'P5', 'P6']

for file_dir in sorted(Path('data/clean_data/').glob('*.fif')):
    subject, task = re.search('(.*)_ses-01_task-(.*)_proc-clean_epo', file_dir.stem).groups()
    
    # open data
    epochs = mne.read_epochs(file_dir, verbose=0)
    epochs.pick_channels(target_ch)
    
    # epoch to continuous
    ch_names = epochs.ch_names
    data = np.hstack(epochs.get_data())
    info = mne.create_info(ch_names,
                       1000,
                       len(ch_names)*['eeg'],
                       )
    raw = mne.io.RawArray(data, info)
    raw.save(f'data/Data4MATLAB_PLI/{subject}_{task}_raw.fif')

## Open mat file and create datatable for wPLI connectivities

In [21]:
all_wpli = loadmat('all_wpli.mat')

# create a list of files
trial_list = []
for file_dir in sorted(Path('data/clean_data/').glob('*.fif')):
    subject, task = re.search('(.*)_ses-01_task-(.*)_proc-clean_epo', file_dir.stem).groups()
    trial_list.append(subject + '_' + task)

wpli_agg = {}
for i in range(len(trial_list)):
    wpli_agg[trial_list[i]] = all_wpli['all_conns'][0][0][i].mean(1)
    
# connectivity labels
conn_info = all_wpli['all_conns'][0][0][-1][0][0][0]

conn_labels = []
for i in range(len(conn_info)):
    conn_labels.append(conn_info[i][0][0] + '_' + conn_info[i][1][0])

conn_df = pd.DataFrame.from_dict(wpli_agg, columns=conn_labels, orient='index')

numpy.ndarray

In [234]:
# create labels for frontoparietal connections
inter_conn = [i
             for i in conn_labels
             if i[0] + i[3] == 'PP' or i[0] + i[3] == 'FF']
ind = [np.where(np.array(conn_labels) == i)[0][0] for i in inter_conn]
frontal_parietal = np.delete(conn_labels, ind)

frontal_parietal = {
'left-frontal_left-pariental' : ['P3_F3', 'P1_F3', 'P5_F3', 'F1_P3', 'F5_P3', 'P1_F1', 'P5_F1', 'P1_F5', 'P5_F5'],
'left-frontal_right-pariental': ['P4_F3', 'P6_F3', 'P2_F3', 'F1_P4', 'F5_P4', 'P2_F1', 'P6_F1', 'P2_F5', 'P6_F5'],
'right-frontal_left-pariental' : ['P3_F4', 'P5_F4', 'F2_P3', 'P1_F2', 'P5_F2', 'P1_F6', 'P5_F6', 'P1_F4', 'F6_P3'],
'right-frontal_right-pariental' : ['P4_F4', 'P2_F4', 'P6_F4', 'F2_P4', 'F6_P4', 'P2_F2', 'P6_F2', 'P2_F6', 'P6_F6']
}

# aggregate connections between frontal and parietal areas
fp_df = pd.DataFrame()
for k,v in frontal_parietal.items():
    fp_df[k] = conn_df[v].mean(1)

# create labels for connections between traget channels
target_ch = ['P4', 'P3', 'F4', 'F3']
target_connections = pd.DataFrame(columns=target_ch, index=target_ch)
target_connections = target_connections.apply(lambda x: x.index + '_' + x.name)
target_connections = target_connections.values[np.triu_indices(target_connections.shape[0], k=1)]

# join aggregated connectivities and target channel connectivities
df = fp_df.join(conn_df[target_connections])

In [ ]:
# Merge with behavioral data
# open behavioral data and ids map
bh = pd.read_csv('data/behavioral_data/archived/plb_hyp_data.csv', index_col='index')
ids_map = pd.read_excel('docs/ids_map.xlsx', header=1, index_col='behavioral_id')
ids_map = ids_map.drop_duplicates('bids_id')
ids_map = ids_map[['bids_id']]
ids_map['bids_id'] = ids_map['bids_id'].apply(lambda x:str(x).zfill(2))
bh = bh.join(ids_map, how='right')
bh = bh.melt(
    id_vars=['procedure_type_1', 'procedure_type_2', 'procedure_type_3', 'procedure_type_4', 'bids_id',
             'description_type_1', 'description_type_2', 'description_type_3', 'description_type_4'],
    value_vars=['hypnosis_depth_1', 'hypnosis_depth_2', 'hypnosis_depth_3', 'hypnosis_depth_4'])
bh['session'] = bh['variable'].apply(lambda x:x.split('_')[2])
bh['procedure'] = bh.apply(lambda r: r['procedure_type_'+r['session']], axis=1)
bh['description'] = bh.apply(lambda r: r['description_type_'+r['session']], axis=1)
bh = bh[['bids_id', 'value', 'procedure', 'description', 'session']].sort_values(by=['bids_id', 'session']).set_index('bids_id')
bh = bh.rename(columns={'value':'hypnosis_depth'})
bh.reset_index(inplace=True)

df[['bids_id', 'condition']] = df.index.to_series().apply(lambda x:x.split('_')).apply(pd.Series)
df['session'] = df['condition'].apply(lambda x:x[-1])
df.reset_index(drop=True, inplace=True)
df['bids_id'] = df['bids_id'].apply(lambda x:x[4:])
df = pd.merge(bh, df, how='right', on=['session', 'bids_id'], right_index=False)
df = df.sort_values(by=['bids_id', 'session', 'condition']).reset_index(drop=True)
df.insert(1, 'condition', df.pop('condition'))
df.head()

# correct for procedure, description, and hypnosis depth of baseline rows
df.loc[df['condition'] == 'baseline1', ['procedure', 'description']] = 'baseline'
df.loc[df['condition'] == 'baseline2', ['procedure', 'description']] = 'baseline'
df.loc[df['condition'] == 'baseline1', ['hypnosis_depth']] = None
df.loc[df['condition'] == 'baseline2', ['hypnosis_depth']] = None

# save
# df.to_csv('wpli_alpha2.csv', index=0)

# Python
## wPLI at sensor level

In [51]:
# constants
freqs = np.arange(1, 42, 1)
foi = np.array([
        [1, 4],
        [4, 8],
        [8, 13],
        [13, 30],
        [30, 42]
        ])
fmin = tuple(foi[:, 0])
fmax = tuple(foi[:, 1])

## open a sample eeg data and create connectivity labels based on its channel names
epochs = mne.read_epochs('data/clean_data/sub-01_ses-01_task-experience1_proc-clean_epo.fif',
                         verbose=0)
epochs.drop_channels(['M1', 'M2'])
epochs.pick_types(eeg=True)
ch_names = epochs.ch_names

conn_labels = pd.DataFrame(columns=ch_names, index=ch_names)
conn_labels = conn_labels.apply(lambda x: x.index + '\N{left right arrow}' + x.name)
conn_labels = conn_labels.values[np.tril_indices(conn_labels.shape[0], k=-1)]

## bands
bands = ['delta', 'theta', 'alpha', 'beta', 'gamma']

# define a function that get a numpy array and return a dataframe of FCs
def calculate_wpli(data, freqs, fmin, fmax, bands, subject, task, conn_labels, sfreq,
                   verbose=0, decim=20):
  conn = spectral_connectivity_time(data,
                                    freqs=freqs,
                                    method='wpli',
                                    fmin=fmin,
                                    fmax=fmax,
                                    sfreq=sfreq,
                                    faverage=True,
                                    average=True,
                                    verbose=verbose,
                                    decim=decim,
                                    n_cycles=5
                                  )

  # get data
  conn = conn.get_data(output='dense')
  
  # create a dataframe from a dictionary of connectivities in different bands
  temp = {}
  for i, b in enumerate(bands):
      temp[b] = conn[:, :, i][np.tril_indices(conn.shape[0], k=-1)]

  df = pd.DataFrame(temp, index=conn_labels).stack().reset_index()
  df['conn'] = df['level_0'] + '_' + df['level_1']
  
  return df.drop(columns=['level_0', 'level_1']).set_index('conn').T.set_index([pd.Index([subject + '_' + task])])

In [ ]:
df_total = pd.DataFrame([]) # connectivity collectors

for subject_path in sorted(Path('data/clean_data').glob('*.fif')):
    subject, task = re.search('(.*)_ses-01_task-(.*)_proc-clean_epo.*', subject_path.stem).groups()
    
    if 'experience' in task:
        # open data and modify
        print(f'>>>>>>>>>>>>>> Processing {subject} -- {task}')
        epochs = mne.read_epochs(subject_path, verbose=0)
        epochs.drop_channels(['M1', 'M2'])
        epochs.pick_types(eeg=True)
        data = np.array(np.split(np.hstack(epochs.get_data()), 10, axis=1))
        
        df = calculate_wpli(data, freqs, fmin, fmax, bands, subject, task, sfreq=1000)
        
        df_total = pd.concat([df_total, df])
        del df

In [ ]:
# behavioral data
bh = pd.read_csv('data/behavioral_data/behavioral_data.csv')[:208]
bh['bids_id'] = bh['bids_id'].astype('int64')
bh['session'] = bh['session'].astype('int64')

# append connectivity and behavioral data 
df_total[['bids_id', 'condition']] = df_total.index.to_series().apply(lambda x:x.split('_')).apply(pd.Series)
df_total['session'] = df_total['condition'].apply(lambda x:int(x[-1]))
df_total['bids_id'] = df_total['bids_id'].apply(lambda x:int(x[4:]))

df_total.reset_index(drop=True, inplace=True)
df_total = pd.merge(bh, df_total, how='right', on=['session', 'bids_id'], right_index=False)
df_total = df_total.sort_values(by=['bids_id', 'session', 'condition']).reset_index(drop=True)
df_total.insert(1, 'condition', df_total.pop('condition'))
# df_total.to_csv('data/wpli_sensor.csv')

## wPLI at source level

In [42]:
# create connectivity labels
# labels
yeo7 = {
    'N1': 'Visual',
    'N2': 'Somatomotor',
    'N3': 'DorsalAttention',
    'N4': 'VentralAttention',
    'N5': 'Limbic',
    'N6': 'Frontoparietal',
    'N7': 'Default',
    'mwall': 'Medial_Wall',
}

hemisferes = ['lh', 'rh']

# labels based on Yeo2011 atlas orders
networks = [yeo7[k]+'_'+ hemisferes[i] for k in yeo7.keys() for i in range(len(hemisferes))]

conn_labels = pd.DataFrame(index=networks, columns=networks)
conn_labels = conn_labels.apply(lambda x: x.index + '\N{left right arrow}' + x.name)
conn_labels = conn_labels.values[np.tril_indices(conn_labels.shape[0], k=-1)]

In [ ]:
df_total = pd.DataFrame([]) # connectivity collectors
for subject_path in sorted(Path('data/parcellated_source_yeo7').glob('*.npz')):
    subject, task = re.search('(.*)_task-(.*)_labels.*', subject_path.stem).groups()
    
    if 'experience' in task:
        
        label_ts = np.load(subject_path)['labels']
        data = np.hstack(label_ts)
        
        # To get 30-second time segments, we need to split source arrays into 10 segments.
        # To make the array divisible by 10, we cut off a few data points (less than 10) at the end:
        idx = data.shape[1] - data.shape[1]%10
        data = data[:, :idx]
        data = np.array(np.split(data, 10, axis=1))
        
        print(f'>>>>>>>>>>>>> Processing {subject} --- {task}')
        # calculate wpli
        df = calculate_wpli(data, freqs, fmin, fmax, bands, subject=subject, task=task, conn_labels=conn_labels,
                            verbose=0, decim=10, sfreq=512)
        
        # append wpli
        df_total = pd.concat([df_total, df])
        del df

In [ ]:
# prepare data for classification
# behavioral data
bh = pd.read_csv('data/behavioral_data/behavioral_data.csv')[:208]
bh['bids_id'] = bh['bids_id'].astype('int64')
bh['session'] = bh['session'].astype('int64')

# append connectivity and behavioral data 
df_total[['bids_id', 'condition']] = df_total.index.to_series().apply(lambda x:x.split('_')).apply(pd.Series)
df_total['session'] = df_total['condition'].apply(lambda x:int(x[-1]))
df_total['bids_id'] = df_total['bids_id'].apply(lambda x:int(x[4:]))

df_total.reset_index(drop=True, inplace=True)
df_total = pd.merge(bh, df_total, how='right', on=['session', 'bids_id'], right_index=False)
df_total = df_total.sort_values(by=['bids_id', 'session', 'condition']).reset_index(drop=True)
df_total.insert(1, 'condition', df_total.pop('condition'))
df_total.head()
# df_total.to_csv('data/classification_datasets/wpli_source.csv')